In [1]:
%load_ext autoreload
%autoreload 2

In [92]:
import os
from datetime import datetime

import polars as pl
from freezegun import freeze_time

from src.activities import (
    get_activities_df,
    get_day_of_week_summaries,
    get_weekly_summaries,
)
from pydantic import BaseModel
from src.constants import COACH_ROLE
from typing import List
from src.auth_manager import get_strava_client
from src.email_manager import send_email, training_week_to_html
from src.supabase_client import upsert_training_week_with_coaching, get_training_week_with_coaching
from src.training_week_generation.training_week import generate_training_week
from src.types.training_week import TrainingSession

# Sunday Night Training Week Message

In [62]:
@freeze_time("2024-08-11 20:00:00")
def get_activities_df_wrapper(strava_client):
    return get_activities_df(strava_client)

client_preferences = "A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays"
sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferenced: {client_preferences}\n"

# activities setup
athlete_id = os.environ["JAMIES_ATHLETE_ID"]
strava_client = get_strava_client(athlete_id)
activities_df = get_activities_df_wrapper(strava_client)

# gen training week pipeline
day_of_week_summaries = get_day_of_week_summaries(activities_df)
weekly_summaries = get_weekly_summaries(activities_df)
training_week_with_coaching = generate_training_week(
    sysmsg_base=sysmsg_base,
    weekly_summaries=weekly_summaries,
    day_of_week_summaries=day_of_week_summaries,
)

# save data to db and trigger email
upsert_training_week_with_coaching(
    athlete_id=athlete_id, training_week_with_coaching=training_week_with_coaching
)
send_email(
    subject="Training Schedule Just Dropped 🏃",
    html_content=training_week_to_html(training_week_with_coaching),
)

athlete_id='98390356' token still valid until 2024-08-17 05:16:28+00:00


{'message_id': '<202408162339.82734117080@smtp-relay.mailin.fr>',
 'message_ids': None}

# Mid-Week Update

In [59]:
@freeze_time("2024-08-13 20:00:00")
def mock_get_activities_df(strava_client, num_weeks=8):
    return get_activities_df(strava_client, num_weeks)


@freeze_time("2024-08-13 20:00:00")
def mock_time_now():
    return datetime.now()


def get_day_of_week():
    return mock_time_now().strftime("%A")


def get_weekday_num():
    return mock_time_now().weekday()


class ActivitySummary(BaseModel):
    date_and_time: str
    """Datetime formatted as 'Monday, August 13, 2024 08:00 PM'"""

    distance_in_miles: float
    elevation_gain_in_feet: float
    pace_minutes_per_mile: float


def get_activity_summaries(strava_client, num_weeks=8) -> pl.DataFrame:
    df = mock_get_activities_df(strava_client, num_weeks)
    concise_activities_df = df.with_columns(
        pl.col('start_date_local').apply(lambda x: x.strftime("%A, %B %d, %Y %I:%M %p")).alias('date_and_time'),
        pl.col('distance_in_miles').apply(lambda x: round(x, 2)),
        pl.col('elevation_gain_in_feet').apply(lambda x: round(x, 2)),
        pl.col('pace_minutes_per_mile').apply(lambda x: round(x, 2)),
    ).drop('id', 'name', 'day_of_week', 'week_of_year', 'year', 'moving_time_in_minutes', 'start_date_local')    
    return [ActivitySummary(**row) for row in concise_activities_df.rows(named=True)]

In [47]:
athlete_id = os.environ["JAMIES_ATHLETE_ID"]
strava_client = get_strava_client(athlete_id)
training_week_with_coaching = get_training_week_with_coaching(athlete_id)
activities = get_activity_summaries(strava_client, num_weeks=1)

athlete_id='98390356' token still valid until 2024-08-17 22:38:15+00:00


In [77]:

class MidWeekAnalysis(BaseModel):
    activities: list[ActivitySummary]
    training_week: List[TrainingSession]
    
    @property
    def training_week_future(self):
        return self.training_week[get_weekday_num() + 1:]
    
    @property
    def miles_ran(self):
        return sum(activity.distance_in_miles for activity in self.activities)
    
    @property
    def miles_target(self):
        return sum(session.distance for session in self.training_week)
    
    @property
    def miles_remaining(self):
        return self.miles_target - self.miles_ran
    
    @property
    def future_miles_planned(self):
        return sum(session.distance for session in self.training_week_future)

In [80]:
mid_week_analysis = MidWeekAnalysis(activities=activities, training_week=training_week_with_coaching.training_week)

In [94]:
from src.llm import get_completion_json
from src.types.training_week import TrainingWeekWithPlanning

client_preferences = "A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays"
sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferenced: {client_preferences}\n"

msg = sysmsg_base + f"""\nThe original plan for this week targeted {mid_week_analysis.miles_target} miles. Your client has run {mid_week_analysis.miles_ran} miles so far (and they have {mid_week_analysis.miles_remaining} miles left to run this week), but the plan for the rest of the week includes {mid_week_analysis.future_miles_planned} miles. Adjust the plan accordingly given the following information:

Here is your client's progress so far this week:
{mid_week_analysis.activities}

Here is the plan for the rest of the week:
{mid_week_analysis.training_week_future}

What changes need to be made so that your client runs {mid_week_analysis.miles_remaining} more miles this week?"""

response = get_completion_json(
        message=msg,
        response_model=TrainingWeekWithPlanning,
    )